In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
tw = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
tw_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
tw.head()

In [ ]:
tw.info()

In [ ]:
tw.target.value_counts(normalize=True)

EDA on keyword

In [ ]:
keyword_list = tw.keyword.unique().tolist()

In [ ]:
target_rate = []
for kw in tw.keyword.unique():
    target_rate.append(tw[tw.keyword==kw].target.mean())

In [ ]:
for kw in keyword_list:
    print(kw,':')
    print(tw[tw.keyword==kw].target.mean())
    print('----')

In [ ]:
keyword_df = pd.DataFrame({
    'keyword':keyword_list[1:],
    'target_rate':target_rate[1:]
})
keyword_df
keyword_df.sort_values(by='target_rate', ascending=False)[:20]

EDA on location

In [ ]:
tw.location.isnull().sum()

In [ ]:
tw.location.value_counts(ascending=False)

In [ ]:
tw[pd.notna(tw.keyword)]

Tokenization

In [ ]:
tw.text[:5]

In [ ]:
# convery tweets to lowercase
tw['clean_text'] = tw['text'].str.lower()

In [ ]:
# remove non-alphanumeric characteristics
import re
tw['clean_text'] = tw['clean_text'].str.replace('\W+', ' ')

In [ ]:
# tokenize
import nltk
tw['clean_text'] = tw['clean_text'].apply(nltk.word_tokenize)

In [ ]:
# stem
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

tw['clean_text'] = tw['clean_text'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
tw.head()

In [ ]:
# use TfidfVectorizer to convert sentences to features. It will also remove stopwords.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english', use_idf = False, norm = None, binary = True, lowercase = True)

X = tfidf.fit_transform(tw['clean_text'].str.join(' '))


In [ ]:
X

In [ ]:
y = tw['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score


clfrf = RandomForestClassifier(n_estimators = 10)
clfrf.fit(X_train, y_train)
y_predrf = clfrf.predict(X_test)
f1rf = f1_score(y_test, y_predrf, average = "macro")
accuracyrf = accuracy_score(y_test, y_predrf)
print("Random forest f1 score:", f1rf, "accuracy:", accuracyrf)

In [ ]:
# feature importance
pd.Series(index=tfidf.get_feature_names(), data=clfrf.feature_importances_).sort_values(ascending=False)[:20]

In [ ]:
# gridsearch

from sklearn.model_selection import GridSearchCV

grid_params = [ { 'n_estimators':[100,200] }]

clf = GridSearchCV(RandomForestClassifier(), grid_params, cv = 3, scoring='f1_macro')
clf.fit(X_train, y_train)

print("Best parameter values: %r\n" % clf.best_params_)
print("Grid scores:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f, stdev %.3f on %r" % (mean, std, params))

In [ ]:
tw_test.head()

In [ ]:
tw_test['clean_text'] = tw_test['text'].str.lower()
tw_test['clean_text'] = tw_test['clean_text'].str.replace('\W+', ' ')
tw_test['clean_text'] = tw_test['clean_text'].apply(nltk.word_tokenize)

stemmer = PorterStemmer()
tw_test['clean_text'] = tw_test['clean_text'].apply(lambda x: [stemmer.stem(word) for word in x])

X_tw_test = tfidf.transform(tw_test['clean_text'].str.join(' '))

In [ ]:
tw_test['target'] = clfrf.predict(X_tw_test)
tw_test = tw_test[['id', 'target']]


In [ ]:
tw_test.head()

In [ ]:
tw_test.to_csv('submission_bow_v1', index=False)